**1. Importing basic libraries & reading data**

In [ ]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../input/WA_Fn-UseC_-Telco-Customer-Churn.csv")


**2. Describing & preprocessing data**

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

*It seems there is no missing value but, I also checked data variety by using value_counts method for each feature. I will not show all of them to save space and time but I will show couple of normal results and 1 irregular result at "Total Charges" column*

In [ ]:
df['SeniorCitizen'].value_counts()

In [ ]:
df['MultipleLines'].value_counts()

In [ ]:
df['TotalCharges'].value_counts()

*As it can be seen, "TotalCharges" column contains 11 values with " ". As they are low in number, let's just get rid of those rows*

In [ ]:
df.replace(" ", np.nan, inplace = True)
df.dropna(subset=["TotalCharges"], axis=0, inplace = True)
df.reset_index(drop = True, inplace = True)

In [ ]:
df.shape

*Now let's transform all categorical values to numerical* **NOTE: Somehow it did not allow me to do it all at once, so doing it in parts!!!**

In [ ]:
cleanup_nums = {"PhoneService": {"Yes": 1, "No": 0}, "PaperlessBilling": {"Yes": 1, "No": 0}, "Churn": {"Yes": 1, "No": 0}}

In [ ]:
df.replace(cleanup_nums, inplace=True)
df.head()

In [ ]:
cleanup_nums = {"MultipleLines": {"Yes": 1, "No": 0, "No phone service": 2}}

In [ ]:
df.replace(cleanup_nums, inplace=True)
df.head()

In [ ]:
cleanup_nums = {"InternetService": {"Fiber optic": 1, "DSL": 2, "No": 0}}

In [ ]:
df.replace(cleanup_nums, inplace=True)
df.head()

In [ ]:
cleanup_nums = {"OnlineSecurity": {"Yes": 1, "No": 0, "No internet service": 2}, "DeviceProtection": {"Yes": 1, "No": 0, "No internet service": 2}, "TechSupport": {"Yes": 1, "No": 0, "No internet service": 2}, "StreamingTV": {"Yes": 1, "No": 0, "No internet service": 2}, "StreamingMovies": {"Yes": 1, "No": 0, "No internet service": 2}, "Contract": {"Month-to-month": 1, "Two year": 3, "One year": 2}, "PaymentMethod": {"Electronic check": 1, "Mailed check": 2, "Bank transfer (automatic)": 3, "Credit card (automatic)": 4}}

In [ ]:
df.replace(cleanup_nums, inplace=True)
df.head()

In [ ]:
cleanup_nums = {"OnlineBackup": {"Yes": 1, "No": 0, "No internet service": 2}}

In [ ]:
df.replace(cleanup_nums, inplace=True)
df.head()

In [ ]:
cleanup_nums = {"gender": {"Male": 1, "Female": 0}, "Partner": {"Yes": 1, "No": 0}, "Dependents": {"Yes": 1, "No": 0}}

In [ ]:
df.replace(cleanup_nums, inplace=True)
df.head()

*Checking correlation among all variables to see if it is ok to apply Logistic Regression (if there is no very high correlation among independent variables, Logistic Regression may not work)*


In [ ]:
import seaborn as sns
plt.figure(figsize=(12,7))
sns.heatmap(cbar=True,annot=True,data=df.corr()*100,cmap='Greens')
plt.title('% Correlation Matrix')
plt.show()


![](http://)*As some of the services here are directly correlated with the customer's having internet services, they yielded relatively high correlation among eachother. Still at this version, I will apply Logistic Regression with all variables and see what happens*

**2. Defining X - Y and applying Logistic Regression**

In [ ]:
X = np.asarray(df[['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', "InternetService", "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod", "MonthlyCharges", "TotalCharges"]])
X[0:5]

In [ ]:
Y = np.asarray(df['Churn'])
Y [0:5]

*Normalizing some of the variables before applying the model*

In [ ]:
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

*Splitting training and testing set*

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  Y_train.shape)
print ('Test set:', X_test.shape,  Y_test.shape)

*Applying the model first and predicting with it*

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,Y_train)
LR

In [ ]:
yhat = LR.predict(X_test)
yhat

yhat_prob = LR.predict_proba(X_test)
yhat_prob

**3. Evaluating the model**

*Testing with jaccard*

In [ ]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(Y_test, yhat)

*Does not seem too bad with 0.80, though can be improved*

*Testing with Confusion Matrix*

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
 
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cnf_matrix = confusion_matrix(Y_test, yhat, labels=[1,0])
np.set_printoptions(precision=2)

In [ ]:
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['churn=1','churn=0'],normalize= True,  title='Confusion matrix')

*Looks ok for predicting customers with no churn, but absolutely not ok for predicting customers with churn. Definitely need improvement*

*Testing with F-score*

In [ ]:
print (classification_report(Y_test, yhat))

*F-score is quite ok with 0.80, though not close to perfect. Details of it also indicate that prediction is much better for no churn customers.*

*Testing with log-loss*

In [ ]:
from sklearn.metrics import log_loss
log_loss(Y_test, yhat_prob)

*log-loss score is really bad with only 0.42*

***So this was the first version and we see that the model needs big improvement. I will continue working on it in later Versions.***